In [365]:
import pandas as pd
from AUIDFunctions import *
uh = pd.read_csv("input/unit-history_demo.csv")
am = pd.read_csv("input/all-members_demo.csv")
ac = pd.read_csv("input/accused_demo.csv")
am = am.rename(columns = {'TID' : 'ID1'})
uh = uh.rename(columns = {'TID' : 'ID2'})

In [368]:
def intersect(a, b):
     return list(set(a) & set(b))
def setdiff(a,b):
    return list(set(a) - set(b))
def union(a,b):
    return list(set(a) | set(b))

In [369]:
TakeFirst4 = lambda x: x[:4]
BY_to_CA = lambda x: 2016 - x
def AddColumns(df, add_cols = ["F4FN", "F4LN", "Current.Age", "CA_BY"]):
    if "F4FN" in add_cols and "First.Name" in df.columns:
            df['F4FN'] = df['First.Name'].map(TakeFirst4)
    if "F4LN" in add_cols and 'Last.Name' in df.columns:
            df['F4LN'] = df['Last.Name'].map(TakeFirst4)
    if "Current.Age" in add_cols and "Current.Age" in df.columns:
        df['Current.Age.p1'] = df['Current.Age']
        df['Current.Age.m1'] = df['Current.Age']
    if "CA_BY" in add_cols and "Birth.Year" in df.columns:
        by_to_ca = lambda x: 2016 - x
        df['Current.Age.p1'] = df['Birth.Year'].map(BY_to_CA)
        df['Current.Age.m1'] = df['Birth.Year'].map(BY_to_CA) - 1  
    return df



In [370]:
amd = am.dropna(axis=1, how='all')
uhd = uh.dropna(axis=1, how='all')
amd = AddColumns(amd)
uhd = AddColumns(uhd)
print(amd.columns)
print(uhd.columns)
cols = [col for col in amd.columns if col in uhd.columns]
print(cols)

Index(['ID1', 'First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name',
       'Appointed.Date', 'Gender', 'Race', 'Birth.Year', 'F4FN', 'F4LN',
       'Current.Age.p1', 'Current.Age.m1'],
      dtype='object')
Index(['ID2', 'First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name',
       'Appointed.Date', 'Current.Age', 'Gender', 'Race', 'Star1', 'Star2',
       'Star3', 'Star4', 'Star5', 'Star6', 'Star7', 'Star8', 'Star9', 'Star10',
       'Current.Unit', 'F4FN', 'F4LN', 'Current.Age.p1', 'Current.Age.m1'],
      dtype='object')
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'F4FN', 'F4LN', 'Current.Age.p1', 'Current.Age.m1']


In [371]:
mc = ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"]
amdmp = RemoveDuplicates(amdm)
uhdmp = RemoveDuplicates(uhdm)
t1 = amdmp.merge(uhdmp, on = mc, how ='inner')

mc = ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"]
amdmp = RemoveDuplicates(amdm)
uhdmp = RemoveDuplicates(uhdm)
t2 = amdmp.merge(uhdmp, on = mc, how ='inner')

print(intersect(t1['ID2'], t2['ID2']))
print(intersect(t1['ID1'], t2['ID1']))
print(t1.shape)
print(t2.shape)

[]
[]
(1312, 16)
(4289, 16)


In [372]:
def LoopMerge(df1, df2, on_list, keep_columns = ['ID1', 'ID2']):
    dfm = pd.DataFrame(columns = keep_columns + ['Match'])
    # Generate on list?
    for mc in on_list:
        df1t = RemoveDuplicates(df1[keep_columns[:1] + mc], mc)
        df2t = RemoveDuplicates(df2[keep_columns[1:] + mc], mc)
        dfmt = df1t.merge(df2t, on=mc, how='inner')
        if dfmt.shape[0] > 0:
            print('******')
            print(mc)
            print(dfmt.shape[0])
            dfmt['Match'] = '-'.join(mc)
            dfm = dfm.append(dfmt[keep_columns + ['Match']].reset_index(drop=True))
            df1 = df1.loc[~df1[keep_columns[0]].isin(dfm[keep_columns[0]])]
            df2 = df2.loc[~df2[keep_columns[1]].isin(dfm[keep_columns[1]])]

    print(df1.shape[0])
    print(df2.shape[0])
    return (dfm.reset_index(drop=True), df1, df2)



In [373]:
m = LoopMerge(amdm, uhdm, on_list = [
    ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['F4FN', 'F4LN', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['F4FN', 'F4LN', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['F4FN', 'F4LN', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['F4FN', 'F4LN', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['F4FN', 'F4LN', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['F4FN', 'F4LN', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name',"Suffix.Name", "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN',"Suffix.Name", "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name',"Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN',"Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Suffix.Name", 'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', "Suffix.Name", 'Appointed.Date', 'Gender', "Race"],
    ['First.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"]
])

******
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
4289
******
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.p1']
1312
******
['F4FN', 'F4LN', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
1
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
19949
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.p1']
6011
******
['F4FN', 'F4LN', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
3
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Middle.Initial', 'Appointed.Date', 'Gender', 'Race']
1
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race']
2
******
['First.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
4
******
['First

In [374]:
def AppendToReference(ref, df):
    return 0

In [375]:
uid_ref = pd.concat([m[0][["ID1", "ID2"]], m[1][["ID1"]], m[2][["ID2"]]]).reset_index(drop=True)
if "UID" not in uid_ref.columns:
    uid_ref.insert(0, "UID", uid_ref.index + 1)

In [376]:
am = am.merge(uid_ref[["ID1", "UID"]], on='ID1', how='left')
uh = uh.merge(uid_ref[["ID2", "UID"]], on='ID2', how='left')

In [389]:
ref = pd.concat([am, uh])

,Appointed.Date,Birth.Year,Current.Age,Current.Unit,First.Name,Gender,ID1,ID2,Last.Name,Middle.Initial,...,Star2,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Suffix.Name,UID
0,2005-09-26,1971.0,NaN,NaN,JEFFERY,MALE,1,NaN,AARON,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5603
1,2005-09-26,1980.0,NaN,NaN,KARINA,FEMALE,2,NaN,AARON,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,1970-06-15,1942.0,NaN,NaN,DANIEL,MALE,3,NaN,ABATE,P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25552
3,1994-12-05,1968.0,NaN,NaN,ANTHONY,MALE,4,NaN,ABBATE,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5604
4,1969-01-06,1942.0,NaN,NaN,CARMEL,MALE,5,NaN,ABBATE,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5605


In [391]:
profiles = AggregateData(ref, uid = "UID", id_cols = [],
              max_cols = setdiff(ref.columns, ["UID"]))

In [392]:
profiles.columns

Index(['UID', 'Star10', 'Star1', 'Current.Age', 'Birth.Year', 'Current.Unit',
       'Last.Name', 'Star3', 'Star8', 'ID2', 'Star5', 'Star4', 'First.Name',
       'Suffix.Name', 'Star9', 'Star6', 'Gender', 'Star7', 'Appointed.Date',
       'ID1', 'Star2', 'Race'],
      dtype='object')

In [387]:
acd = ac.dropna(axis=1, how='all')
acd = AddColumns(acd)
print(acd.columns)
print(profiles.columns)

Index(['TID', 'First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name',
       'Appointed.Date', 'Birth.Year', 'Gender', 'Race', 'Current.Unit',
       'Current.Rank', 'Star', 'F4FN', 'F4LN', 'Current.Age.p1',
       'Current.Age.m1'],
      dtype='object')
Index(['UID', 'Star10', 'Star1', 'Current.Age', 'Birth.Year', 'Current.Unit',
       'Last.Name', 'Star3', 'Star8', 'ID2', 'Star5', 'Star4', 'First.Name',
       'Suffix.Name', 'Star9', 'Star6', 'Gender', 'Star7', 'Appointed.Date',
       'ID1', 'Star2', 'Race'],
      dtype='object')


In [385]:
m = LoopMerge(profiles, acd, on_list = [
    ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', "Suffix.Name",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name',"Suffix.Name", "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN',"Suffix.Name", "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name',"Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN',"Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Suffix.Name", 'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', "Suffix.Name", 'Appointed.Date', 'Gender', "Race"],
    ['First.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race"]
], keep_columns = ["UID", "TID"])

KeyError: "['Middle.Initial'] not in index"